In [22]:
!pip install texthero

In [3]:
import os
from typing import Dict

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.models import model_from_json

import texthero as hero

from texthero import stopwords

from nltk.corpus import stopwords

In [4]:
input_file = 'D:\\repos\\ml_crash_course\\data\\labeled_data_corpus.csv'
if not os.path.exists(input_file):
    raise RuntimeError(f'No input file: {input_file}')

df = pd.read_csv(input_file)
train_df = df[df['subset'] == 'train']
test_df = df[df['subset'] == 'test']
print('num rows for train: %d', train_df.shape[0])


num rows for train: %d 5233


In [5]:
X_train = train_df['msg']
y_train = train_df['label']

X_test = test_df['msg']
y_test = test_df['label']

In [6]:
X_train[:10]

0    здравствуйте. ишу 2х спальную квартиру в лимас...
1    #сниму  комнату в лимассоле или недалеко от не...
2                        мошенник риэлторским услугам.
3    **sales**    reg.1053 lic.489/e **stylish apar...
4    важно: [valerii korol](tg://user?id=193474890)...
5    аренда  no: 367/e ️ларнака️между пила и декели...
6    привет  ищу виллу посуточно с бюджетом 2000€ в...
7    важно: [liss](tg://user?id=202814885), если ты...
8                               total messages: 126772
9    аренда  ️ларнака ️в центре города️ saint lazar...
Name: msg, dtype: object

In [7]:
X_train = X_train.str.lower()
X_train = X_train.dropna()

X_train = hero.remove_digits(X_train)
X_train = hero.remove_punctuation(X_train)
X_train = hero.remove_stopwords(X_train)
X_train = hero.remove_whitespace(X_train)

russian_stopwords = stopwords.words("russian")
X_train = hero.remove_stopwords(X_train, russian_stopwords)

greek_stopwords = stopwords.words("greek")
X_train = hero.remove_stopwords(X_train, greek_stopwords)

turkish_stopwords = stopwords.words("turkish")
X_train = hero.remove_stopwords(X_train, turkish_stopwords)

X_train = hero.remove_whitespace(X_train)

#only utf8 allowed
X_train = X_train.map(lambda x: bytes(x, 'utf-8').decode('utf-8', 'ignore'))

In [8]:
X_train[:10]

0    здравствуйте ишу 2х спальную квартиру лимассол...
1    сниму комнату лимассоле недалеко начала август...
2                         мошенник риэлторским услугам
3    sales reg lic e stylish apartment sea view kis...
4    важно valerii korol tg user id бот спамер прой...
5    аренда e ларнакамежду пила декелия пешей досту...
6    привет ищу виллу посуточно бюджетом сутки дней...
7    важно liss tg user id бот спамер пройди провер...
8                                       total messages
9    аренда ларнака центре города saint lazaro chur...
Name: msg, dtype: object

In [9]:
unique_words = list(X_train.str.split(' ', expand=True).stack().unique())
len(unique_words)

14795

In [10]:
a = X_train.apply(lambda x: len(x.split(" ")))

In [11]:
a.sum() / len(a)

34.68087139308236

In [12]:
X_test = X_test.str.lower()
X_test = X_test.dropna()

X_test = hero.remove_digits(X_test)
X_test = hero.remove_punctuation(X_test)
X_test = hero.remove_stopwords(X_test)
X_test = hero.remove_whitespace(X_test)

russian_stopwords = stopwords.words("russian")
X_test = hero.remove_stopwords(X_test, russian_stopwords)

greek_stopwords = stopwords.words("greek")
X_test = hero.remove_stopwords(X_test, greek_stopwords)

turkish_stopwords = stopwords.words("turkish")
X_test = hero.remove_stopwords(X_test, turkish_stopwords)

X_test = hero.remove_whitespace(X_test)

#only utf8 allowed
X_test = X_test.map(lambda x: bytes(x, 'utf-8').decode('utf-8', 'ignore'))

In [13]:
X_test[:10]

5233    сдам собственник лимассол сдаю комнатную новую...
5234    важно tiana tg user id бот спамер пройди прове...
5235    привет алена tg user id это бесплатная группа ...
5236    привет meds tg user id это бесплатная группа а...
5237    аренда район линопетра лимассол долгосрочную а...
5238                                     клиент приезжает
5239                    james michael tg user id забанили
5240    сдается квартира пафосе район хлорака м2 спаль...
5241    аренда х комнатного дома джакузи выходом крышу...
5242    квартира спальнями ванной комнатой верандами к...
Name: msg, dtype: object

In [14]:
X_train = X_train.values

X_test = X_test.values

In [15]:
len(X_train)

5233

In [16]:
X_train[0]

'здравствуйте ишу 2х спальную квартиру лимассоле желательно гермасойя семья 2х взрослых 2х детей животных длительный срок бюджет евро предложения лс'

In [17]:
VOCAB_SIZE = 14000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(X_train)

In [18]:
vocab = np.array(encoder.get_vocabulary())
len(vocab[:])

14000

In [19]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    
    tf.keras.layers.TextVectorization(
        max_tokens=VOCAB_SIZE,output_mode='int',
        vocabulary=np.delete(np.delete(vocab, 0),0)),
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=32,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
])

In [20]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, None, 32)          448000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 466,753
Trainable params: 466,753
Non-trai

In [21]:
history4 = model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=20)

Epoch 1/20
164/164 [==============================] - 16s 61ms/step - loss: 0.6316 - precision: 0.3137 - recall: 0.0153 - val_loss: 0.4993 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
164/164 [==============================] - 8s 46ms/step - loss: 0.3828 - precision: 0.9667 - recall: 0.1106 - val_loss: 0.2918 - val_precision: 0.9412 - val_recall: 0.4867
Epoch 3/20
164/164 [==============================] - 8s 47ms/step - loss: 0.2320 - precision: 0.8924 - recall: 0.7350 - val_loss: 0.2282 - val_precision: 0.8577 - val_recall: 0.8023
Epoch 4/20
164/164 [==============================] - 8s 47ms/step - loss: 0.1679 - precision: 0.8647 - recall: 0.9075 - val_loss: 0.2128 - val_precision: 0.8654 - val_recall: 0.8555
Epoch 5/20
164/164 [==============================] - 8s 49ms/step - loss: 0.1314 - precision: 0.8754 - recall: 0.9447 - val_loss: 0.2052 - val_precision: 0.8633 - val_recall: 0.8403
Epoch 6/20
164/164 [==============================] - 8s 47ms/step - loss: 0

In [29]:
precision = history4.history['val_precision'][-1]

In [30]:
recall = history4.history['val_recall'][-1]

In [31]:
f1 = (2 * recall * precision) / (recall + precision)
print(f1)

0.8816793864275645


In [32]:
# serialize model to JSON
model_json = model.to_json()
with open("D:\\repos\\ml_crash_course\\data\\model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to tf
model.save_weights("D:\\repos\\ml_crash_course\\data\\model", save_format = 'tf')
print("Saved model to disk")

Saved model to disk


In [33]:
# load json and create model
json_file = open("D:\\repos\\ml_crash_course\\data\\model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("D:\\repos\\ml_crash_course\\data\\model")
print("Loaded model from disk")

Loaded model from disk


In [34]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, None, 32)          448000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 466,753
Trainable params: 466,753
Non-trai

In [35]:
def preprocessing(messages : np.ndarray) -> pd.Series:
    pd_messages = pd.Series(messages)
    pd_messages = pd_messages.str.lower()
    pd_messages = pd_messages.dropna()

    pd_messages = hero.remove_digits(pd_messages)
    pd_messages = hero.remove_punctuation(pd_messages)
    pd_messages = hero.remove_stopwords(pd_messages)
    pd_messages = hero.remove_whitespace(pd_messages)

    russian_stopwords = stopwords.words("russian")
    pd_messages = hero.remove_stopwords(pd_messages, russian_stopwords)

    greek_stopwords = stopwords.words("greek")
    pd_messages = hero.remove_stopwords(pd_messages, greek_stopwords)

    turkish_stopwords = stopwords.words("turkish")
    pd_messages = hero.remove_stopwords(pd_messages, turkish_stopwords)

    pd_messages = hero.remove_whitespace(pd_messages)

    return pd_messages

In [40]:
msg = '#аренда #квартира  #лимассол #агент.  ппрп: квартира / 1 спальня    абобус -в конце макариос авеню - апостолос андреас  трехэтажный жилой дом, расположенный в тихом, спокойном жилом районе лимассола, в непосредственной близости от центра города, с легким доступом к близлежащим школам, супермаркетам, а также многим другим услугам , новый порт , молл. квартира с 1 спальней  - открытого плана зал совмещен с кухней  душевая комната  и просторная веранда  крытая парковка и кладовая   все здание теплоизолированное с использованием энергосберегающих технологий класса а, а все окна оснащены двойным тепловым остеклением. квартира сдается  без мебели, будет  оборудована электроприборами и установлены кондиционеры. идеальное тихое место в центре города! цена снижена : €1,050 /per month (plus 1 deposit )'


print(preprocessing(np.array( [msg] ))[0])

pred = loaded_model.predict(preprocessing(np.array( [msg] )))
print(pred[0])

аренда квартира лимассол агент ппрп квартира спальня абобус конце макариос авеню апостолос андреас трехэтажный жилой дом расположенный тихом спокойном жилом районе лимассола непосредственной близости центра города легким доступом близлежащим школам супермаркетам также многим другим услугам новый порт молл квартира спальней открытого плана зал совмещен кухней душевая комната просторная веранда крытая парковка кладовая здание теплоизолированное использованием энергосберегающих технологий класса окна оснащены двойным тепловым остеклением квартира сдается мебели оборудована электроприборами установлены кондиционеры идеальное тихое место центре города цена снижена per month plus deposit
[0.999206]


In [ ]:
# serialize model to JSON

model_path = conf.model_path
weights_path = conf.weights_path

model_json = model.to_json()

with open(model_path, "w") as json_file:
    json_file.write(model_json)
# serialize weights to TF
model.save_weights(weights_path)
print("Saved model to disk")

In [42]:
a= [{'name': 'Homer', 'age': 39}, {'name': 'Bart', 'age': 10}]
newlist = sorted(a, key=lambda d: d['age']) 
print(newlist)

[{'name': 'Bart', 'age': 10}, {'name': 'Homer', 'age': 39}]
